In [4]:
from abc import ABCMeta, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem(object):

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order(object):

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount= self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(object):
    __metaclass__ = ABCMeta

    @abstractmethod
    def discount(self, order):
        pass


class FidelityPromotion(Promotion):

    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromotion(Promotion):

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.01
        return discount


class LargeOrderPromotion(Promotion):

    def discount(self, order):
        discount_items = {item.product for item in order.cart}
        if len(discount_items) >= 10:
            return order.total() * 0.07
        return 0

In [5]:
joe = Customer('John Doe', 0)

In [6]:
ann = Customer('Ann Smith', 1100)

In [7]:
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermelon', 5, 5.0)]

In [8]:
Order(joe, cart, FidelityPromotion())

<Order total: 42.00 due: 42.00>

In [9]:
Order(ann, cart, FidelityPromotion())

<Order total: 42.00 due: 39.90>

In [10]:
class Order(object):

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount= self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


def fidelity_discount(order):
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0


def bulk_item_discount(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.01
    return discount


def large_order_discount(order):
    discount_items = {item.product for item in order.cart}
    if len(discount_items) >= 10:
        return order.total() * 0.07
    return 0

In [11]:
Order(joe, cart, fidelity_discount)

<Order total: 42.00 due: 42.00>

In [12]:
Order(ann, cart, fidelity_discount)

<Order total: 42.00 due: 39.90>